<a href="https://colab.research.google.com/github/sebastian-dv/CSE-151A-Project/blob/main/Seb_Model3_SVM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 151A Group Project
Names
Pranav Prabu
Sebastian Diaz
Jou-Chih Chang
Juan Yin
Irving Zhao
Xianzhe Guo
Tiankuo Li

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [136]:
df = pd.read_csv('https://archive.ics.uci.edu/static/public/880/data.csv')
df = df[['age','sex','death','dzgroup','scoma','race','sps','aps','diabetes','dementia','meanbp','wblc','hrt','resp','temp','pafi','alb','bili','crea','sod','ph']]

In [34]:
df.head()

,age,sex,death,dzgroup,scoma,race,sps,aps,diabetes,dementia,...,wblc,hrt,resp,temp,pafi,alb,bili,crea,sod,ph
0,62.84998,male,0,Lung Cancer,0.0,other,33.898438,20.0,0,0,...,6.000000,69.0,22.0,36.00000,388.00000,1.799805,0.199982,1.199951,141.0,7.459961
1,60.33899,female,1,Cirrhosis,44.0,white,52.695312,74.0,0,0,...,17.097656,112.0,34.0,34.59375,98.00000,NaN,NaN,5.500000,132.0,7.250000
2,52.74698,female,1,Cirrhosis,0.0,white,20.500000,45.0,0,0,...,8.500000,88.0,28.0,37.39844,231.65625,NaN,2.199707,2.000000,134.0,7.459961
3,42.38498,female,1,Lung Cancer,0.0,white,20.097656,19.0,0,0,...,9.099609,88.0,32.0,35.00000,NaN,NaN,NaN,0.799927,139.0,NaN
4,79.88495,female,0,ARF/MOSF w/Sepsis,26.0,white,23.500000,30.0,0,0,...,13.500000,112.0,20.0,37.89844,173.31250,NaN,NaN,0.799927,143.0,7.509766


## Preprocessing

In [137]:
from sklearn.preprocessing import StandardScaler

# droping nan values
df = df.dropna(axis = 0, how = 'any')

# scaling
scaler = StandardScaler()
categorical = ['sex', 'dzgroup', 'race', 'death', 'diabetes', 'dementia']
df_cat = df[categorical]
df_num = df.drop(categorical, axis = 1)
df_num = pd.DataFrame(scaler.fit_transform(df_num), columns = df_num.columns)

df_num.reset_index(drop = True, inplace = True)
df_cat.reset_index(drop = True, inplace = True)

df = pd.concat([df_num, df_cat], axis = 1)
target = df['dzgroup']

# one hot encoding race
ohe = pd.get_dummies(df[['race']])
df = df.drop(['race'], axis = 1)
df = pd.concat([df, ohe], axis = 1)

# encoding sex
df['sex'].replace('female', 0, inplace=True)
df['sex'].replace('male', 1, inplace=True)

# copying pre-ohe dzgroup
df_pre_ohe = df.copy()

# one hot encoding dzgroup
ohe = pd.get_dummies(df[['dzgroup']])
df = df.drop(['dzgroup'], axis = 1)

X = df.copy()
y = ohe.copy()
df = pd.concat([df, ohe], axis = 1)

In [101]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [102]:
# 1. Build a linear kernel SVM
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# C is the SVM regularization parameter
C = 0.1  # able to change

# Create an Instance of SVM and Fit out the data.
# Data is not scaled so as to be able to plot the support vectors
svm = SVC(kernel ='linear')
svm.fit(X_train,y_train.idxmax(axis=1).values)
y_true = y_test.idxmax(axis=1).values
y_pred = svm.predict(X_test)
print(classification_report(y_true, y_pred))
# Evaluate the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

                           precision    recall  f1-score   support

dzgroup_ARF/MOSF w/Sepsis       0.56      0.89      0.69       340
              dzgroup_CHF       0.45      0.36      0.40        95
             dzgroup_COPD       0.42      0.30      0.35        93
        dzgroup_Cirrhosis       0.00      0.00      0.00        52
     dzgroup_Colon Cancer       0.00      0.00      0.00        14
             dzgroup_Coma       0.73      0.53      0.62        62
      dzgroup_Lung Cancer       0.44      0.47      0.45        43
     dzgroup_MOSF w/Malig       0.00      0.00      0.00        66

                 accuracy                           0.54       765
                macro avg       0.33      0.32      0.31       765
             weighted avg       0.44      0.54      0.47       765

Accuracy: 0.5437908496732026


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Hyperparameter Tuning


In [103]:
# find the beter parameters
# test for best model
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}

# Create an SVM classifier
svm_classifier = SVC()

# Create the GridSearchCV object
grid_search = GridSearchCV(svm_classifier, param_grid, cv=3, scoring='accuracy')

# Fit the model with different hyperparameter combinations
grid_search.fit(X_train,y_train.idxmax(axis=1).values)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
accuracy = best_model.score(X_test, y_test.idxmax(axis=1).values)
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy on Test Set: {accuracy}")

Best Hyperparameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy on Test Set: 0.5568627450980392


## Oversampling Data


### RandomOverSampler

In [155]:
X2 = df_pre_ohe.drop('dzgroup', axis = 1)
y2 = df_pre_ohe['dzgroup']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=21)

In [156]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

rs = RandomOverSampler(random_state=11)
X2_train_res, y2_train_res = rs.fit_resample(X2_train, y2_train)

# Results
print("Before Oversampling: \n", y2_train.value_counts(), "\n")
print("After Oversampling: \n", y2_train_res.value_counts())

Before Oversampling: 
 ARF/MOSF w/Sepsis    1377
CHF                   377
COPD                  353
MOSF w/Malig          269
Coma                  202
Lung Cancer           202
Cirrhosis             195
Colon Cancer           84
Name: dzgroup, dtype: int64 

After Oversampling: 
 COPD                 1377
ARF/MOSF w/Sepsis    1377
Coma                 1377
CHF                  1377
MOSF w/Malig         1377
Lung Cancer          1377
Cirrhosis            1377
Colon Cancer         1377
Name: dzgroup, dtype: int64


### Evaluation

In [157]:
# ohe
y2_test = pd.get_dummies(y2_test)
y2_train_res = pd.get_dummies(y2_train_res)

In [160]:
svm_classifier = SVC(kernel='rbf', C = 10, gamma = 0.1)
svm_classifier.fit(X2_train_res, y2_train_res.idxmax(axis = 1).values)
y2_true = y2_test.idxmax(axis=1).values
y2_pred = svm_classifier.predict(X2_test)
print(classification_report(y2_true, y2_pred))

# Evaluate the accuracy
accuracy = accuracy_score(y2_true, y2_pred)
print(f"Accuracy: {accuracy}")

                   precision    recall  f1-score   support

ARF/MOSF w/Sepsis       0.61      0.77      0.68       340
              CHF       0.42      0.37      0.39        95
             COPD       0.41      0.35      0.38        93
        Cirrhosis       0.33      0.19      0.24        52
     Colon Cancer       0.07      0.07      0.07        14
             Coma       0.71      0.55      0.62        62
      Lung Cancer       0.38      0.28      0.32        43
     MOSF w/Malig       0.30      0.20      0.24        66

         accuracy                           0.52       765
        macro avg       0.40      0.35      0.37       765
     weighted avg       0.50      0.52      0.50       765

Accuracy: 0.5241830065359477


In [ ]:
# find the beter parameters
# test for best model
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}

# Create an SVM classifier
svm_classifier = SVC()

# Create the GridSearchCV object
grid_search = GridSearchCV(svm_classifier, param_grid, cv=3, scoring='accuracy')

# Fit the model with different hyperparameter combinations
grid_search.fit(X2_train_res,y2_train_res.idxmax(axis=1).values)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Get the best model
best_model = grid_search.best_estimator_

In [165]:
# Evaluate the best model on the test set
accuracy = best_model.score(X2_test, y2_test.idxmax(axis=1).values)
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy on Test Set: {accuracy}")

Best Hyperparameters: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Accuracy on Test Set: 0.45098039215686275
